In [3]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

Nous souhaitons ici représenter chaque buisness avec un score sur 5 labels : food, inside, outside, menu et drink. Pour ca il faut définir des axes sémantiques pour chaque cluster.

In [44]:
food_axes = {
    'luxury': 'artistic gourmet plated dish with fine presentation',
    'touristic': 'typical american dish simple presentation',
    'hype': 'colorful trendy instagrammable food presentation',
    'fastfood': 'fast food burger fries and soda',
    'local': 'traditional homestyle authentic cuisine',
    'casual': 'comfort food simple generous portion',
    'hidden': 'high quality dish'
}
food_labels = {
    'luxury': 'luxury gourmet food',
    'touristic': 'touristic food',
    'hype': 'instagrammable food',
    'fastfood': 'fast food food',
    'local': 'traditional homestyle cuisine',
    'casual': 'casual comfort food',
    'hidden': 'good and simple dish'
}

inside_axes = {
    'luxury': 'elegant fine dining restaurant interior with white tablecloths',
    'touristic': 'tourist-oriented restaurant decor',
    'hype': 'trendy aesthetically designed restaurant interior',
    'fastfood': 'fast food chain restaurant counter',
    'local': 'authentic neighborhood restaurant cozy family atmosphere',
    'casual': 'simple comfortable restaurant interior',
    'hidden': 'simple quality-focused restaurant interior'
}

outside_axes = {
    'luxury': 'upscale restaurant elegant sophisticated facade',
    'touristic': 'tourist-oriented restaurant with visible signage',
    'hype': 'stylish aesthetically designed storefront with people waiting',
    'fastfood': 'recognizable chain restaurant brand exterior',
    'local': 'traditional neighborhood local restaurant',
    'casual': 'welcoming informal restaurant entrance',
    'hidden': 'discreet unassuming small restaurant'
}

drink_axes = {
    'luxury': 'craft cocktail sophisticated wine presentation',
    'touristic': 'standard beverage simple presentation',
    'hype': 'aesthetically presented photogenic colorful drink',
    'fastfood': 'soda fountain soft drink',
    'local': 'traditional local beverage or beer',
    'casual': 'casual drinks',
    'hidden': 'quality beverage selection'
}

menu_axes = {
    'luxury': 'refined curated short menu',
    'touristic': 'extensive menu with pictures of typical dishes',
    'hype': 'modern trendy menu design',
    'fastfood': 'standardized chain menu board',
    'local': 'traditional simple menu, can be handwritte',
    'casual': 'simple straightforward menu',
    'hidden': 'small specialized unique menu'
}

In [ ]:
import os 
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '300'
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors=True)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image = Image.open("/Users/teatoscanduplantier/Desktop/ENSAE/Machine_Learning/Projet/Machine-learning-for-python/Etude_photos/photos/-aVTOdp5TYbDhOfgJwRQ0g.jpg") 
texts = list(food_labels.values())
single_text = 'an inside view of a restaurant'
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

scores_bruts = logits_per_image[0].numpy()
scores = probs[0].numpy()

In [46]:
cluster_scores = {}
for i, cluster in enumerate(food_labels.keys()):
    cluster_scores[cluster] = scores_bruts[i]
print(cluster_scores)

{'luxury': 25.388752, 'touristic': 24.883623, 'hype': 21.372309, 'fastfood': 25.303658, 'local': 25.860052, 'casual': 23.488832, 'hidden': 24.397402}


Après avoir analysé plusieurs photos, nous constatons que cette méthode n'est pas efficace, donc nous ne continuons pas sur cette piste